In [ ]:
import numpy as np
import os
import networkx as nx
from src.MOF_builder._readcif import process_node,read_cif,extract_atoms_ccoords_from_lines,extract_type_atoms_ccoords_in_primitive_cell
from src.MOF_builder._learn_template import make_supercell_3x3x3,find_pair_v_e,extract_unit_cell
from src.MOF_builder._learn_template import add_ccoords,set_DV_V,set_DE_E
from src.MOF_builder._readcif import extract_type_atoms_fcoords_in_primitive_cell

node_cif = 'node1.cif'
linker_cif = 'diedge.cif'

##linker
#linker_cell_info,_, linker_atom_site_sector = read_cif(linker_cif)
#_,_, linker_x_ccoords=extract_type_atoms_ccoords_in_primitive_cell(linker_cif, 'X')
##ditopic linker only has two x vectors
##linker_length = calc_edge_len(linker_cif,'.') #length of the edge should be x-x length in linker cif file, unit angstrom
#linker_length = np.linalg.norm(linker_x_ccoords[0]-linker_x_ccoords[1])
#linker_cell_info,_, linker_atom_site_sector = read_cif(linker_cif)
#ll,linker_atom, linker_ccoords = extract_atoms_ccoords_from_lines(linker_cell_info,linker_atom_site_sector)
#print(linker_length,'linker_length')
#
#node
node_target_type = 'Zr'
#node_unit_cell,node_atom, node_x_fcoords, node_fcoords = process_node(node_cif, node_target_type)



#template cif 
template_cif = 'fcu.cif'
template_cell_info,_,vvnode = extract_type_atoms_fcoords_in_primitive_cell(template_cif, 'V')
_,_,eenode = extract_type_atoms_fcoords_in_primitive_cell(template_cif, 'E')
unit_cell = extract_unit_cell(template_cell_info)

vvnode = np.unique(np.array(vvnode,dtype=float),axis=0)
eenode = np.unique(np.array(eenode,dtype=float),axis=0)
##loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
vvnode333 = make_supercell_3x3x3(vvnode)
eenode333 = make_supercell_3x3x3(eenode)

pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
G = add_ccoords(G,unit_cell)



G = set_DV_V(G)
G = set_DE_E(G)

def debug_xxxx(xxxx_positons_dict):
    with open("xxxx_positions.xyz", "w") as file:
        num_atoms = sum(len(positions) for positions in xxxx_positons_dict.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in xxxx_positons_dict.items():
            for pos in positions:
                file.write(f"X{str(node):4s}   {pos[1]:8.3f} {pos[2]:8.3f} {pos[3]:8.3f}\n")


 no symmetry operation
 no symmetry operation
V0 [0. 0. 0.] 12
V V0 [0. 0. 0.]
V1 [0.  0.5 0.5] 12
V V1 [0.  0.5 0.5]
V36 [0. 0. 1.] 2
V12 [0. 1. 0.] 2
V48 [0. 1. 1.] 1
V2 [0.5 0.  0.5] 12
V V2 [0.5 0.  0.5]
V3 [0.5 0.5 0. ] 12
V V3 [0.5 0.5 0. ]
V39 [0.5 0.5 1. ] 2
V14 [0.5 1.  0.5] 2
V4 [1. 0. 0.] 2
V40 [1. 0. 1.] 1
V5 [1.  0.5 0.5] 2
V20 [1. 1. 0.] 1
V10 [-0.5  0.   0.5] 2
V11 [-0.5  0.5  0. ] 2
V47 [-0.5  0.5  1. ] 1
V30 [-0.5  1.   0.5] 1
V17 [ 0.  -0.5  0.5] 2
V19 [ 0.5 -0.5  0. ] 2
V55 [ 0.5 -0.5  1. ] 1
V25 [ 1.  -0.5  0.5] 1
V35 [-0.5 -0.5  0. ] 1
V73 [ 0.   0.5 -0.5] 2
V74 [ 0.5  0.  -0.5] 2
V86 [ 0.5  1.  -0.5] 1
V77 [ 1.   0.5 -0.5] 1
V82 [-0.5  0.  -0.5] 1
V89 [ 0.  -0.5 -0.5] 1


In [2]:
from scipy.spatial.transform import Rotation
from scipy.optimize import minimize



def apply_rotation(q, p):
    """ Apply quaternion rotation to point p """
    r = Rotation.from_quat(q)
    return r.apply(p)

def objective(q_flat, G, static_atom_positions, sorted_nodes,sorted_edges):
    """ Compute sum of distances between connected clusters after rotation """
    node_num = G.number_of_nodes()
    q_matrix_d1 = q_flat.reshape((1, 4))
    q_matrix_d1 /= np.linalg.norm(q_matrix_d1, axis=1, keepdims=True)  # Normalize
    #each q should be same
    q_matrix = np.tile(q_matrix_d1,node_num).reshape((node_num, 4))

    total_distance = 0

    for (i, j) in sorted_edges:
        R_i = Rotation.from_quat(q_matrix[i]).as_matrix()
        R_j = Rotation.from_quat(q_matrix[j]).as_matrix()

        com_i = G.nodes[sorted_nodes[i]]['ccoords']
        com_j = G.nodes[sorted_nodes[j]]['ccoords']

        # Rotate positions around their mass center
        rotated_i_positions = np.dot(static_atom_positions[i][:,1:] - com_i, R_i.T) + com_i
        rotated_j_positions = np.dot(static_atom_positions[j][:,1:] - com_j, R_j.T) + com_j

        dist_matrix = np.empty((len(rotated_i_positions), len(rotated_j_positions)))    
        for idx_i in range(len(rotated_i_positions)):
            for idx_j in range(len(rotated_j_positions)):
                dist = np.linalg.norm(rotated_i_positions[idx_i] - rotated_j_positions[idx_j])
                dist_matrix[idx_i, idx_j] = dist
                total_distance += (dist ** 2)
        #for idx_i in range(len(rotated_i_positions)):
        #    diff = max(dist_matrix[idx_i,:]) - min(dist_matrix[idx_i, :])
        #    if diff > 1:
        #        total_distance += 10000/diff 
        #    else:#penalty for the distance difference
        #        total_distance += 1e5
        #for idx_j in range(len(rotated_j_positions)):
        #    diff = max(dist_matrix[:,idx_j]) - min(dist_matrix[:, idx_j])
        #    if diff > 1:
        #        total_distance += 10000/diff 
        #    else:#penalty for the distance difference
        #        total_distance += 1e5

    return total_distance


def optimize_q_rotations(num_nodes,G,sorted_nodes,sorted_edges,atom_positions,opt_methods="L-BFGS-B",maxfun=15000):
    """
    Optimize rotations for all nodes in the graph.

    Parameters:
        G (networkx.Graph): Graph structure with edges between nodes.
        atom_positions (dict): Positions of X atoms for each node.

    Returns:
        list: Optimized rotation matrices for all nodes.
    """
        # Initial random rotations (quaternions)
    node_num = G.number_of_nodes()
    #initial_quaternions = np.random.randn(node_num, 4)
    #initial_quaternions /= np.linalg.norm(initial_quaternions, axis=1, keepdims=True)  # Normalize


    #initial_quaternions = np.([0, 1, 0, 0], num_nodes)
    #initial_quaternions = np.tile([1, 0, 0, 0], 1) #every node are the same 
    initial_quaternions = np.random.randn(1, 4)
    initial_quaternions /= np.linalg.norm(initial_quaternions, axis=1, keepdims=True)  # Normalize

    #get a better initial guess, use random rotation matrix combination
    #initial_rotations  = np.array([reorthogonalize_matrix(np.random.rand(3,3)) for i in range(num_nodes)]).flatten()
    static_atom_positions = atom_positions.copy()
    # Precompute edge-specific pairings
    #edge_pairings = find_edge_pairings(sorted_edges, atom_positions)

    result = minimize(
        objective,
        initial_quaternions.flatten(),  # Flatten the initial quaternions to 1D
        args=(G, static_atom_positions, sorted_nodes, sorted_edges),
        method=opt_methods,
        options={"maxiter": 5000, "disp": True, "maxfun": maxfun},
    )
    
    # Extract optimized quaternions
    #optimized_quaternions should be the same for all nodes, so we need to tile it
    optimized_quaternion = result.x.reshape((1, 4))
    optimized_quaternions = np.tile(optimized_quaternion,node_num).reshape((node_num, 4))


    #optimized_quaternions = result.x.reshape((node_num, 4))

    print("Optimized Rotations (Quaternions):", optimized_quaternions)

## # Print the optimized pairings after optimization
## print("Optimized Pairings (after optimization):")
## for (i, j), pairs in edge_pairings.items():
##     print(f"Node {i} and Node {j}:")
##     for idx_i, idx_j in pairs:
##         print(f"  node{i}_{idx_i} -- node{j}_{idx_j}")
## print()

    return optimized_quaternions,static_atom_positions



In [3]:
#ERROR comes from the PBC condition of template node. it overcross the boundary, and dot unit cell before COM it will be wrong

In [ ]:
from src.MOF_builder._node_rotation_matrix_optimizer import optimize_rotations,apply_rotations_to_atom_positions,apply_rotations_to_xxxx_positions,update_ccoords_by_optimized_cell_params
from src.MOF_builder._scale_cif_optimizer import optimize_cell_parameters
from src.MOF_builder._place_node_edge import addidx,get_edge_lengths,update_node_ccoords,unit_cell_to_cartesian_matrix,cartesian_to_fractional
from src.MOF_builder._superimpose import superimpose
import re

def save_xyz(filename, rotated_positions_dict):
    """
    Save the rotated positions to an XYZ file for visualization.
    """
    with open(filename, "w") as file:
        num_atoms = sum(len(positions) for positions in rotated_positions_dict.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in rotated_positions_dict.items():
            for pos in positions:
                file.write(f"X{node}   {pos[0]:.8f} {pos[1]:.8f} {pos[2]:.8f}\n")
                
def is_list_A_in_B(A,B):
        return all([np.allclose(a,b,atol=0.05) for a,b in zip(A,B)])


# Function to fetch indices and coordinates of atoms with a specific label
def fetch_X_atoms_ind_array(array, column, X):
	# array: input array
	# column: column index to check for label
	# X: label to search for

	ind = [k for k in range(len(array)) if re.sub(r'\d', '', array[k, column]) == X]
	x_array = array[ind]
	return ind, x_array

def check_inside_unit_cell(point):
    return all([i>=-0.01 and i<=1.01 for i in point])

def find_pair_v_e_c(vvnode333, ecnode333,eenode333): #exist center of linker  in mof 
    G = nx.Graph()
    pair_ve = []
    for e in eenode333:
        #print(e,'check')
        dist_v_e = np.linalg.norm(vvnode333 - e, axis=1)
        # find two v which are nearest to e, and at least one v is in [0,1] unit cell
        v1 = vvnode333[np.argmin(dist_v_e)]
        v1_idx = np.argmin(dist_v_e)
        dist_c_e = np.linalg.norm(ecnode333 - e, axis=1)
        # find two v which are nearest to e, and at least one v is in [0,1] unit cell
        v2 = ecnode333[np.argmin(dist_c_e)]
        v2_idx = np.argmin(dist_c_e)
        #print(v1,v2,'v1,v2')

        # find the center of the pair of v
        center = (v1 + v2) / 2
        # check if there is a v in [0,1] unit cell
        if check_inside_unit_cell(v1) or check_inside_unit_cell(v2):
            # check if the center of the pair of v is around e
            #if abs(np.linalg.norm(v1 - e)+np.linalg.norm(v2 - e) - np.linalg.norm(v1 - v2))< 1e-2: #v1,v2,e are colinear
            if np.linalg.norm(center - e) < 0.1:
                #print(e,v1,v2,'check')
                G.add_node('V'+str(v1_idx), fcoords=v1,note='V')
                G.add_node('CV'+str(v2_idx), fcoords=v2,note ='CV')
                G.add_edge('V'+str(v1_idx), 'CV'+str(v2_idx), fcoords=(v1, v2),fc_center=e),
                pair_ve.append(('V'+str(v1_idx), 'CV'+str(v2_idx), e))
    return pair_ve, len(pair_ve), G

def sort_nodes_by_type_connectivity(G):
    Vnodes = [n for n in G.nodes() if G.nodes[n]['type']=='V']
    DVnodes = [n for n in G.nodes() if G.nodes[n]['type']=='DV']
    Vnodes = sorted(Vnodes,key=lambda x: G.degree(x),reverse=True)
    DVnodes = sorted(DVnodes,key=lambda x: G.degree(x),reverse=True)
    return Vnodes+DVnodes

def check_edge_inunitcell(G,e):
    if 'DV' in G.nodes[e[0]]['type'] or 'DV' in G.nodes[e[1]]['type']:
        return False
    return True

def find_and_sort_edges_bynodeconnectivity(graph, sorted_nodes):
    all_edges = list(graph.edges())

    sorted_edges = []
    #add unit_cell edge first

    ei = 0
    while ei < len(all_edges):
            e = all_edges[ei]
            if check_edge_inunitcell(graph,e):
                sorted_edges.append(e)
                all_edges.pop(ei)
            ei += 1
    #sort edge by sorted_nodes
    for n in sorted_nodes:
        ei = 0
        while ei < len(all_edges):
            e = all_edges[ei]
            if n in e:
                if n ==e[0]:
                    sorted_edges.append(e)
                else:
                    sorted_edges.append((e[1],e[0]))
                all_edges.pop(ei)
            else:
                ei += 1

    return sorted_edges   

class net_optimizer():
    def __init__(self,G):
        self.G = G
    
    def set_template_cell_info(self,cell_info):
        self.template_cell_info = cell_info
    
    def set_constant_length(self,constant_length):
        self.constant_length = constant_length

    def node_info(self,node_cif,node_target_type):
        self.node_cif = node_cif
        self.node_target_type = node_target_type
        self.node_unit_cell,self.node_atom, self.node_x_fcoords, self.node_fcoords = process_node(node_cif, node_target_type)
        self.node_x_ccoords = np.dot(self.node_unit_cell,self.node_x_fcoords.T).T
        self.node_coords = np.dot(self.node_unit_cell,self.node_fcoords.T).T

    def linker_info(self,linker_cif):
        self.linker_cif = linker_cif
        self.linker_unit_cell,self.linker_atom, self.linker_x_fcoords, self.linker_fcoords = process_node(linker_cif, 'X')
        self.linker_x_ccoords = np.dot(self.linker_unit_cell,self.linker_x_fcoords.T).T
        self.linker_length = np.linalg.norm(self.linker_x_ccoords[0]-self.linker_x_ccoords[1])
        linker_ccoords = np.dot(self.linker_unit_cell,self.linker_fcoords.T).T
        self.linker_ccoords = linker_ccoords - np.mean(linker_ccoords,axis=0)

    

    def linker_center(self,linker_center_cif):
        self.linker_center_cif = linker_center_cif
        self.ec_unit_cell,self.ec_atom, self.ec_x_fcoords, self.ec_fcoords = process_node(self.linker_center_cif, 'X')
        self.ec_ccoords = np.dot(self.ec_unit_cell,self.ec_x_fcoords.T).T
        self.eccoords = np.dot(self.ec_unit_cell,self.ec_fcoords.T).T

    def set_maxfun(self,maxfun):
        self.maxfun = maxfun

    def set_opt_method(self,opt_method):
        self.opt_method = opt_method

    def optimize(self):

        node_xcoords = self.node_x_ccoords
        node_coords = self.node_coords
        G = self.G
        linker_length = self.linker_length
        opt_method = self.opt_method
        cell_info = self.template_cell_info

        if not hasattr(self,'constant_length'):
            self.constant_length = 1.54
            constant_length = 1.54
            print('constant_length is not set, use default C-C value 1.54')
        else:
            constant_length = self.constant_length

        if not hasattr(self,'ec_atom'):
            self.ec_atom = None
        if not hasattr(self,'ec_x_ccoords'):
            self.ec_x_ccoords = None
        if not hasattr(self,'ec_fcoords'):
            self.ec_fcoords = None
        if not hasattr(self,'ec_unit_cell'):
            self.ec_unit_cell = None
        if not hasattr(self,'ec_xcoords'):
            self.ec_xcoords = None
            ec_xcoords = None
        if not hasattr(self,'eccoords'):
            self.eccoords = None
            eccoords = None

        sorted_nodes = sort_nodes_by_type_connectivity(G)

        #firstly, check if all V nodes have highest connectivity
        #secondly, sort all DV nodes by connectivity

        sorted_edges = find_and_sort_edges_bynodeconnectivity(G,sorted_nodes)

        nodes_atoms = {}
        for n in sorted_nodes:
            if 'CV' in n:
                nodes_atoms[n]= self.ec_atom
            else:
                nodes_atoms[n]= self.node_atom

        xxxx_positions_dict = {}
        node_positions_dict = {}
        #reindex the nodes in the xxxx_positions with the index in the sorted_nodes, like G has 16 nodes[2,5,7], but the new dictionary should be [0,1,2]
        for n in sorted_nodes:
            if 'CV' in n:
                xxxx_positions_dict[sorted_nodes.index(n)]=addidx(G.nodes[n]['ccoords']+ec_xcoords) 
            else:
                xxxx_positions_dict[sorted_nodes.index(n)]=addidx(G.nodes[n]['ccoords']+node_xcoords) 

        for n in sorted_nodes:
            if 'CV' in n:
                node_positions_dict[sorted_nodes.index(n)]=G.nodes[n]['ccoords']+eccoords
            else:
                node_positions_dict[sorted_nodes.index(n)]=G.nodes[n]['ccoords']+node_coords
                

        #reindex the edges in the G with the index in the sorted_nodes
        sorted_edges_of_sortednodeidx = [(sorted_nodes.index(e[0]),sorted_nodes.index(e[1])) for e in sorted_edges]


        # Optimize rotations
        num_nodes = G.number_of_nodes()

        #
        ##3D free rotation #TODO:DEBUG
        #
        optimized_rotations,static_xxxx_positions = optimize_rotations(num_nodes,G,sorted_nodes,
                                                                            sorted_edges_of_sortednodeidx, 
                                                                            xxxx_positions_dict,opt_method,self.maxfun)
#
        #DEBUG

        ##optimized_quaternions,static_xxxx_positions = optimize_q_rotations(num_nodes,G,sorted_nodes,sorted_edges_of_sortednodeidx,xxxx_positions_dict,opt_method,self.maxfun)
        ##        # make optimized quaternion to rotation matrix
        ##optimized_rotations = []
        ##for q in optimized_quaternions:
        ##    r = Rotation.from_quat(q)
        ##    #make rotation matrix to be orthogonal
        ##    r = r.as_matrix()
        ##    optimized_rotations.append(r)

        # Apply rotations
        rotated_node_positions = apply_rotations_to_atom_positions(optimized_rotations, G, sorted_nodes,node_positions_dict)

        # Save results to XYZ
        save_xyz("optimized_nodesstructure.xyz", rotated_node_positions)

        rotated_xxxx_positions_dict,optimized_pair=apply_rotations_to_xxxx_positions(optimized_rotations,
                                                                                     G, 
                                                                                     sorted_nodes, 
                                                                                     sorted_edges_of_sortednodeidx,
                                                                                     xxxx_positions_dict)

        start_node_a = sorted_edges[0][0]#find_nearest_node_to_beginning_point(G)
        start_node_b = sorted_edges[0][1]#find_nearest_node_to_beginning_point(G)
        start_nodes = [start_node_a,start_node_b]
        #loop all of the edges in G and get the lengths of the edges, length is the distance between the two nodes ccoords
        edge_lengths,lengths = get_edge_lengths(G)

        x_com_length = np.mean([np.linalg.norm(i) for i in node_xcoords])
        new_edge_length = linker_length+2*constant_length+2*x_com_length
        #update the node ccoords in G by loop edge, start from the start_node, and then update the connected node ccoords by the edge length, and update the next node ccords from the updated node

        updated_ccoords,original_ccoords = update_node_ccoords(G,edge_lengths,start_node_a,new_edge_length) #TODO: include more than one start node to avoid only 2 components connected
        updated_ccoords_b,original_ccoord_b = update_node_ccoords(G,edge_lengths,start_node_b,new_edge_length)
        #updated_ccoords = {**updated_ccoords_a,**updated_ccoords_b}
        #original_ccoords= {**original_ccoord_a,**original_ccoord_b}

        #exclude the start_node in updated_ccoords and original_ccoords
        updated_ccoords = {k:v for k,v in updated_ccoords.items() if k not in start_nodes}
        original_ccoords = {k:v for k,v in original_ccoords.items() if k not in start_nodes}


        #use optimized_params to update all of nodes ccoords in G, according to the fccoords

        optimized_params = optimize_cell_parameters(cell_info,original_ccoords,updated_ccoords)
        sc_unit_cell = unit_cell_to_cartesian_matrix(optimized_params[0],optimized_params[1],optimized_params[2],optimized_params[3],optimized_params[4],optimized_params[5])
        sc_unit_cell_inv = np.linalg.inv(sc_unit_cell)
        sG,scaled_ccoords = update_ccoords_by_optimized_cell_params(G,optimized_params)
        scaled_node_positions_dict = {}
        scaled_xxxx_positions_dict = {}

        for n in sorted_nodes:
            if 'CV' in n:
                scaled_xxxx_positions_dict[sorted_nodes.index(n)]=addidx(sG.nodes[n]['ccoords']+ec_xcoords) 
            else:
                scaled_xxxx_positions_dict[sorted_nodes.index(n)]=addidx(sG.nodes[n]['ccoords']+node_xcoords) 

        for n in sorted_nodes:
            if 'CV' in n:
                scaled_node_positions_dict[sorted_nodes.index(n)]=sG.nodes[n]['ccoords']+eccoords
            else:
                scaled_node_positions_dict[sorted_nodes.index(n)]=sG.nodes[n]['ccoords']+node_coords


        # Apply rotations
        scaled_rotated_node_positions  = apply_rotations_to_atom_positions(optimized_rotations, sG, sorted_nodes,scaled_node_positions_dict)
        scaled_rotated_xxxx_positions,optimized_pair = apply_rotations_to_xxxx_positions(optimized_rotations, sG,sorted_nodes, sorted_edges_of_sortednodeidx, scaled_xxxx_positions_dict)
        # Save results to XYZ

        self.sorted_nodes = sorted_nodes
        self.sorted_edges = sorted_edges
        self.sorted_edges_of_sortednodeidx = sorted_edges_of_sortednodeidx
        self.optimized_rotations = optimized_rotations
        self.optimized_params = optimized_params
        self.new_edge_length = new_edge_length
        self.optimized_pair = optimized_pair
        self.scaled_rotated_node_positions = scaled_rotated_node_positions
        self.scaled_rotated_xxxx_positions = scaled_rotated_xxxx_positions
        self.sc_unit_cell = sc_unit_cell
        self.sc_unit_cell_inv = sc_unit_cell_inv
        self.sG_node = sG
        self.nodes_atom = nodes_atoms
        save_xyz("scale_optimized_nodesstructure.xyz", scaled_rotated_node_positions)

    def place_edge_in_net(self):
        #linker_middle_point = np.mean(linker_x_ccoords,axis=0)
        linker_xx_vec = self.linker_x_ccoords
        linker_length = self.linker_length
        optimized_pair = self.optimized_pair
        scaled_rotated_xxxx_positions = self.scaled_rotated_xxxx_positions
        scaled_rotated_node_positions = self.scaled_rotated_node_positions
        sorted_nodes = self.sorted_nodes
        sG_node = self.sG_node
        sc_unit_cell_inv = self.sc_unit_cell_inv
        nodes_atom = self.nodes_atom

        sG = sG_node.copy()
        scalar = (linker_length+2*self.constant_length)/linker_length
        extended_linker_xx_vec = [i*scalar for i in linker_xx_vec]
        norm_xx_vector_record = []
        rot_record = []
        #edges = {}
        for (i,j),pair in optimized_pair.items():
            x_idx_i,x_idx_j = pair
            reindex_i = sorted_nodes.index(i)
            reindex_j = sorted_nodes.index(j)
            x_i = scaled_rotated_xxxx_positions[reindex_i][x_idx_i][1:]
            x_j = scaled_rotated_xxxx_positions[reindex_j][x_idx_j][1:]
            x_i_x_j_middle_point = np.mean([x_i,x_j],axis=0)
            xx_vector = np.vstack([x_i-x_i_x_j_middle_point,x_j-x_i_x_j_middle_point])
            norm_xx_vector = xx_vector/np.linalg.norm(xx_vector)
            
            #print(i,j,reindex_i,reindex_j,x_idx_i,x_idx_j)
            #use superimpose to get the rotation matrix
            #use record to record the rotation matrix for get rid of the repeat calculation
            indices = [index for index, value in enumerate(norm_xx_vector_record) if is_list_A_in_B(norm_xx_vector, value)]
            if len(indices) == 1: 
                rot = rot_record[indices[0]]
                #rot = reorthogonalize_matrix(rot)
            else:
                _, rot, _ = superimpose(extended_linker_xx_vec,xx_vector)
                #rot = reorthogonalize_matrix(rot)
                norm_xx_vector_record.append(norm_xx_vector)
                rot_record.append(rot)
                
            #use the rotation matrix to rotate the linker x coords
            #rotated_xx = np.dot(extended_linker_xx_vec, rot)
            #print(rotated_xx,'rotated_xx',xx_vector) #DEBUG
            placed_edge_ccoords = np.dot(self.linker_ccoords, rot) + x_i_x_j_middle_point

            placed_edge = np.hstack((np.asarray(self.linker_atom), placed_edge_ccoords))
            sG.edges[(i,j)]['coords'] = x_i_x_j_middle_point
            sG.edges[(i,j)]['c_points']=placed_edge
            sG.edges[(i,j)]['f_points'] = np.hstack((placed_edge[:,0:1],cartesian_to_fractional(placed_edge[:,1:4],sc_unit_cell_inv)))
            _,sG.edges[(i,j)]['x_coords'] = fetch_X_atoms_ind_array(placed_edge,0,'X')
            #edges[(i,j)]=placed_edge
        #placed_node = {}
        for k,v in scaled_rotated_node_positions.items():
            #print(k,v)
            #placed_node[k] = np.hstack((nodes_atom[k],v))
            sG.nodes[k]['c_points'] = np.hstack((nodes_atom[k],v))
            sG.nodes[k]['f_points'] = np.hstack((nodes_atom[k],cartesian_to_fractional(v,sc_unit_cell_inv)))
            #find the atoms starts with "x" and extract the coordinates
            _,sG.nodes[k]['x_coords'] = fetch_X_atoms_ind_array(sG.nodes[k]['c_points'],0,'X')
        self.sG = sG
        return sG  
    


In [10]:
fcu = net_optimizer(G)
fcu.set_template_cell_info(template_cell_info)
fcu.set_constant_length(1.0)
fcu.node_info(node_cif,node_target_type)
fcu.linker_info(linker_cif)
fcu.set_maxfun(20000)
fcu.set_opt_method('l-bfgs-b')
fcu.optimize()
fcu.place_edge_in_net()

P1 cell
P1 cell
node_x_fcoords[[ 0.155 -0.105 -0.104]
 [-0.155  0.095 -0.113]
 [ 0.155  0.095 -0.113]
 [-0.155 -0.105 -0.104]
 [-0.155 -0.095  0.113]
 [ 0.155  0.105  0.104]
 [-0.155  0.105  0.104]
 [ 0.155 -0.095  0.113]]
node_ccoords_com[ 0.00191613  0.00388065 -0.00972903]
P1 cell
P1 cell
node_x_fcoords[[ 0.0185 -0.1565 -0.0045]
 [-0.0185  0.1565  0.0045]]
node_ccoords_com[-0.09490217 -0.03018043  0.21753696]
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          252     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.42197D+06    |proj g|=  3.38526D+03


 This problem is unconstrained.



At iterate    1    f=  3.42019D+06    |proj g|=  2.11988D+03

At iterate    2    f=  3.41918D+06    |proj g|=  1.59480D+03

At iterate    3    f=  3.41815D+06    |proj g|=  3.27332D+03

At iterate    4    f=  3.41787D+06    |proj g|=  2.39960D+03

At iterate    5    f=  3.41756D+06    |proj g|=  2.90386D+02

At iterate    6    f=  3.41752D+06    |proj g|=  2.63099D+02

At iterate    7    f=  3.41742D+06    |proj g|=  2.54670D+02

At iterate    8    f=  3.41741D+06    |proj g|=  2.62540D+02

At iterate    9    f=  3.41741D+06    |proj g|=  1.15391D+02

At iterate   10    f=  3.41741D+06    |proj g|=  5.26663D+01

At iterate   11    f=  3.41740D+06    |proj g|=  7.44592D+01

At iterate   12    f=  3.41740D+06    |proj g|=  6.90576D+01

At iterate   13    f=  3.41740D+06    |proj g|=  1.84402D+01

At iterate   14    f=  3.41740D+06    |proj g|=  1.44821D+01

At iterate   15    f=  3.41740D+06    |proj g|=  1.92318D+01

At iterate   16    f=  3.41740D+06    |proj g|=  5.57862D+01

At iter

In [6]:
fcu.scaled_rotated_xxxx_positions,

with open("scaled_rotated_xxxx_positions.xyz", "w") as file:
    num_atoms = sum(len(positions) for positions in fcu.scaled_rotated_xxxx_positions.values())
    file.write(f"{num_atoms}\n")
    file.write("Optimized structure\n")

    for node, positions in fcu.scaled_rotated_xxxx_positions.items():
        for pos in positions:
            file.write(f"X{str(node):4s}   {pos[1]:8.3f} {pos[2]:8.3f} {pos[3]:8.3f}\n")

def debug_xxxx(xxxx_positons_dict):
    with open("1xxxx_positions.xyz", "w") as file:
        num_atoms = sum(len(positions) for positions in xxxx_positons_dict.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in xxxx_positons_dict.items():
            for pos in positions:
                file.write(f"X{str(node):4s}   {pos[1]:8.3f} {pos[2]:8.3f} {pos[3]:8.3f}\n")
debug_xxxx(fcu.scaled_rotated_xxxx_positions)

In [7]:
from src.MOF_builder._place_node_edge import place_edgeinnodeframe
from src.MOF_builder._implement_edges import addxoo2edge
from src.MOF_builder._cluster import cluster_supercell,placed_arr
from src.MOF_builder._output import tempgro,temp_xyz,viewgro#,viewxyz
from src.MOF_builder._terminations import terminate_nodes,terminate_unsaturated_edges,add_node_terminations,exposed_xoo_cc,Xpdb,terminate_unsaturated_edges_CCO2
#from functions.filtX import filt_nodex_fvec,filt_closest_x_angle,filt_outside_edgex
#unctions.multitopic import 
from src.MOF_builder._isolated_node_cleaner import reindex_frag_array,get_frag_centers_fc,calculate_eG_net_ditopic
from src.MOF_builder._replace import fetch_by_idx_resname,sub_pdb
from src.MOF_builder._supercell import Carte_points_generator

####### Global options #######


pi = np.pi

class MOF_ditopic:
    #def __init__(self,template,node,edge,node_topics):
    #    self.template = template
    #    self.node = node
    #    self.edge = edge
    #    self.linker_topics = 2 #ditopic class 
    #    self.node_topics = node_topics
	
    def __init__(self):
        self.linker_topics = 2
        pass
    def load(self,TG,placed_all,sc_unit_cell,placed_nodes,placed_edges):
        self.TG = TG
        self.placed_all = placed_all
        self.sc_unit_cell = sc_unit_cell
        self.placed_nodes = placed_nodes
        self.placed_edges = placed_edges

    def basic_supercell(self,supercell,term_file = 'data/methyl.pdb',boundary_cut_buffer = 0.00,edge_center_check_buffer = 0.0,cutxyz=[True,True,True]):
        linker_topics = self.linker_topics
        cutx,cuty,cutz = cutxyz
        TG = self.TG
        scalar = boundary_cut_buffer
        boundary_scalar = edge_center_check_buffer
        placed_edges = self.placed_edges
        placed_nodes = self.placed_nodes
        sc_unit_cell = self.sc_unit_cell
		
        frame_node_name= list(TG.nodes())
        frame_node_fc=np.asarray([TG.nodes[fn]['fcoords']for fn in frame_node_name])
    
        #new_beginning_fc = find_new_node_beginning(frame_node_fc)		
        new_beginning_fc = np.array([0,0,0])
        placed_nodes_arr,nodes_id=placed_arr(placed_nodes)
        placed_edges_arr,edges_id=placed_arr(placed_edges)		
        placed_nodes_fc = np.hstack((placed_nodes_arr[:,0:1],np.dot(np.linalg.inv(sc_unit_cell),placed_nodes_arr[:,1:4].T).T-new_beginning_fc,placed_nodes_arr[:,4:]))
        placed_edges_fc = np.hstack((placed_edges_arr[:,0:1],np.dot(np.linalg.inv(sc_unit_cell),placed_edges_arr[:,1:4].T).T-new_beginning_fc,placed_edges_arr[:,4:]))		
			
        target_all_fc = np.vstack((placed_nodes_fc,placed_edges_fc))
        #target_all_fc = np.vstack((placed_nodes_fc,tetratopic_edges_fcoords)) # the reason for use above version node is because we need xoo in node for terminations adding
        box_bound= supercell+1
        supercell_Carte = Carte_points_generator(supercell)		
        connected_nodeedge_fc, boundary_connected_nodes_res,eG,bare_nodeedge_fc_loose=cluster_supercell(sc_unit_cell,supercell_Carte,linker_topics,target_all_fc,box_bound,scalar,cutx,cuty,cutz,boundary_scalar)		
        terms_cc_loose = terminate_nodes(term_file,boundary_connected_nodes_res,connected_nodeedge_fc,sc_unit_cell,box_bound)

        connected_nodeedge_cc = np.hstack((connected_nodeedge_fc[:,:-3],np.dot(sc_unit_cell,connected_nodeedge_fc[:,-3:].T).T))
        #print(connected_nodeedge_cc.shape,terms_cc_loose.shape)

        node_edge_term_cc_loose = np.vstack((connected_nodeedge_cc,terms_cc_loose))		
        self.all_connected_node_edge_cc = connected_nodeedge_cc
        self.all_terms_cc_loose = terms_cc_loose
        self.all_N_E_T_cc = node_edge_term_cc_loose		
        self.bare_nodeedge_fc = bare_nodeedge_fc_loose

    def write_basic_supercell(self,gro,xyz):
        tempgro('20test.gro',self.all_connected_node_edge_cc[self.all_connected_node_edge_cc[:,5]==1])
        tempgro(gro,self.all_N_E_T_cc)
        temp_xyz(xyz,self.all_N_E_T_cc)
        viewgro("20test.gro")
        viewgro(gro)
	
	
    '''
        #Defective model: node/edge missing
            #delete nodes or edges
            #terminate nodes ROUND1
            # find main fragment
                # find unstaturated node uN1
                    # find uN1 neighbors and extract X in neighbor edge(E+int)
                        # filt exposed X sites in uN1
                            # add terminations

            #terminate edge ROUND2
                #find unsaturaed edge uE1
                    #find uE1 neighbors and extract X in neighbor node 'int'
                        # filt exposed X sites in uE1
                            # add terminations (-OH)

        #Defective model: linker exchange
        #   termination OO don't change, use X to set a range 
        #   atoms in outX_range stay
        #    then superimpose by X


            #superimpose for replacement
                #find X AND super impose
    '''
    def defect_missing(self,remove_node_list = [],remove_edge_list = []):
        bare_nodeedge_fc_loose = self.bare_nodeedge_fc
        linker_topics = self.linker_topics
        node_topics = self.node_topics
        sc_unit_cell =self.sc_unit_cell

        renode1_fcarr=reindex_frag_array(bare_nodeedge_fc_loose,'NODE')
        reedge1_fcarr=reindex_frag_array(bare_nodeedge_fc_loose,'EDGE')
        defective_node_fcarr = np.vstack(([i for i in renode1_fcarr if i[5] not in remove_node_list]))
        defective_edge_fcarr = np.vstack(([i for i in reedge1_fcarr if i[5] not in remove_edge_list]))
        renode_fcarr = reindex_frag_array(defective_node_fcarr,'NODE')
        reedge_fcarr = reindex_frag_array(defective_edge_fcarr,'EDGE')
        edgefc_centers = get_frag_centers_fc(reedge_fcarr)
        nodefc_centers = get_frag_centers_fc(renode_fcarr)

        #eG = calculate_eG_net_ditopic(edgefc_centers,nodefc_centers,linker_topics)
        eG = calculate_eG_net_ditopic(edgefc_centers,nodefc_centers,linker_topics)
        eG_subparts=[len(c) for c in sorted(nx.connected_components(eG), key=len, reverse=True)]
        #for pillar stack, add virtual edge to connect clostest nodes, set distance to 0.6
        if len(eG_subparts)>1:
            print(f'this MOF has {len(eG_subparts)} seperated fragments: {eG_subparts}')
        else:
            print(f'this MOF has {len(eG_subparts)} fragment')



        frags=[(len(c),c) for c in sorted(nx.connected_components(eG), key=len, reverse=True)]
        main_frag=list(sorted(nx.connected_components(eG), key=len, reverse=True)[0]) 
       

        main_frag_nodes = [i for i in main_frag if isinstance(i,int)]
        main_frag_edges = [i for i in main_frag if re.sub('[0-9]','',str(i)) == 'E']



        ###delete "virtual" edges #next version TODO:
        ##for edge_n in eG.edges():
        ##    if eG.edges[edge_n]['type'] == 'virtual':
        ##        eG.remove_edge(edge_n[0],edge_n[1])

        unsaturated_nodes = [(n,d) for n, d in eG.degree() if d <node_topics and isinstance(n,int)]
        unsaturated_edges = [(n,d) for n, d in eG.degree() if d <linker_topics and isinstance(n,str)]
        


        if len(unsaturated_edges) > 0 :
            print(f"UNsaturated edges(linkers) exist, need linker_termination <= {len(unsaturated_edges)}")
        else:
            print("only saturated edges(linkers) exist")

        if len(unsaturated_nodes) > 0 :
            print(f"UNsaturated nodes exist, <={len(unsaturated_nodes)} nodes need node_termination")
            print(unsaturated_nodes)
        else:
            print("only saturated nodes exist")



        unsaturated_main_frag_nodes = [i for i in unsaturated_nodes if i[0] in main_frag_nodes]
        unsaturated_main_frag_edges = [i for i in unsaturated_edges if i[0] in main_frag_edges]

        main_frag_half_edges_fc = np.vstack(([reedge_fcarr[reedge_fcarr[:,5]==int(ei[1:])]for ei in main_frag_edges]))
        main_frag_nodes_fc = np.vstack(([renode_fcarr[renode_fcarr[:,5]==ni]for ni in main_frag_nodes]))
        main_frag_edges_fc,xoo_dict,con_nodes_x_dict = addxoo2edge(eG,main_frag_nodes,main_frag_nodes_fc,main_frag_edges,main_frag_half_edges_fc,sc_unit_cell)

        main_frag_nodes_cc = np.hstack((main_frag_nodes_fc[:,:-3],np.dot(main_frag_nodes_fc[:,-3:],sc_unit_cell)))
        main_frag_edges_cc = np.hstack((main_frag_edges_fc[:,:-3],np.dot(main_frag_edges_fc[:,-3:],sc_unit_cell)))
        self.eG = eG
        self.main_frag_nodes = main_frag_nodes
        self.main_frag_edges = main_frag_edges
        self.unsaturated_main_frag_nodes = unsaturated_main_frag_nodes
        self.unsaturated_main_frag_edges = unsaturated_main_frag_edges
        self.node_xoo_dict = xoo_dict
        self.con_nodes_x_dict = con_nodes_x_dict
        self.main_frag_nodes_cc = main_frag_nodes_cc
        self.main_frag_edges_cc = main_frag_edges_cc   

    def term_defective_model(self,n_term_file = 'data/methyl.pdb',e_termfile = 'data/CCO2.pdb'):
        eG = self.eG
        unsaturated_main_frag_nodes = self.unsaturated_main_frag_nodes
        main_frag_nodes = self.main_frag_nodes
        main_frag_nodes_cc =self.main_frag_nodes_cc
        con_nodes_x_dict =self.con_nodes_x_dict
        xoo_dict = self.node_xoo_dict
        unsaturated_main_frag_edges = self.unsaturated_main_frag_edges
        main_frag_edges_cc = self.main_frag_edges_cc
        linker_topics = self.linker_topics
		
		# get indices for cleaved node atoms in main_frag_nodes  without xoo

        xoo_ind = []
        for key in list(xoo_dict):
            xoo_ind.append(key)
            xoo_ind += xoo_dict[key]
        single_node = main_frag_nodes_cc[main_frag_nodes_cc[:,5]==main_frag_nodes_cc[0,5]]
        node_nums = len(single_node)
        single_node_stay_ind=np.asarray([i for i in range(node_nums) if i not in xoo_ind])
        a =[]
        for node in range(len(main_frag_nodes)):
            a.append(node_nums*node+single_node_stay_ind)
        metal_node_indices = np.hstack(a)

        #add -methyl to terminate nodes find exposed xoo 
        ex_node_cxo_cc_loose=exposed_xoo_cc(eG,unsaturated_main_frag_nodes,main_frag_nodes_cc,con_nodes_x_dict,xoo_dict)
        n_terms_loose = add_node_terminations(n_term_file,ex_node_cxo_cc_loose)
        if len(n_terms_loose) > 0:
            n_terms_cc = np.vstack((n_terms_loose))
            self.n_terms_cc = n_terms_cc
        else:
            n_terms_cc = np.empty((0,9))
            self.n_terms_cc = n_terms_cc

        #add -COOH term to exposed edge and change edge name to HEDGE
        if os.path.basename(e_termfile)=='CCO2.pdb':
            cleaved_metal_node = main_frag_nodes_cc[metal_node_indices]
            t_edges = terminate_unsaturated_edges_CCO2(e_termfile,unsaturated_main_frag_edges,eG,main_frag_edges_cc,linker_topics)
            node_edge_term_cc= np.vstack((cleaved_metal_node,t_edges,n_terms_cc))
        else:
            cleaved_metal_node = main_frag_nodes_cc[metal_node_indices]
            t_edges = terminate_unsaturated_edges(e_termfile,unsaturated_main_frag_edges,eG,main_frag_edges_cc,linker_topics)
            node_edge_term_cc= np.vstack((cleaved_metal_node,t_edges,n_terms_cc))
        self.t_edges = t_edges
        self.tn_te_cc = node_edge_term_cc
			


    def write_tntemof(self,gro):
        tempgro(gro,self.tn_te_cc)
        #temp_xyz("303term_supercell.xyz",self.tn_te_cc)
        viewgro(gro)


    def defect_replace_linker(self,sub_file,sub_class,candidate_res_idx_list,sub_res_newname = 'SUB'):
        node_edge_term_cc = self.tn_te_cc
        for res_idx in candidate_res_idx_list:
            fetch_res_mask,other_mask = fetch_by_idx_resname(node_edge_term_cc,res_idx,sub_class)
            other_res = node_edge_term_cc[other_mask]
            selected_res = node_edge_term_cc[fetch_res_mask]
            if len(selected_res)==0:
                continue
            X_atoms_ind = [i for i in range(len(selected_res)) if selected_res[i,2][0] == 'X']
            outer_atoms_ind = [j for j in range(len(selected_res)) if (j > X_atoms_ind[0]) & (j not in X_atoms_ind)]
            stay_outer_atoms = selected_res[outer_atoms_ind]
            X_atoms = selected_res[X_atoms_ind]
            sub_data=sub_pdb(sub_file)
            subX,subX_ind= Xpdb(sub_data,'X')
            subX_coords_cc = subX[:,-3:]
            subX_coords_cc = subX_coords_cc.astype('float')
            X_atoms_coords_cc = X_atoms[:,-3:]
            X_atoms_coords_cc =  X_atoms_coords_cc.astype('float')

            _,rot,trans = superimpose(subX_coords_cc,X_atoms_coords_cc)
            sub_coords = sub_data[:,-3:]
            sub_coords = sub_coords.astype('float')
            placed_sub_data = np.hstack((sub_data[:,:-3],np.dot(sub_coords,rot)+trans))

            sub_edge = np.vstack((stay_outer_atoms,placed_sub_data))
            sub_edge[:,5]= stay_outer_atoms[0,5]
            sub_edge[:,4] = sub_res_newname
            for row in range(len(sub_edge)):
                sub_edge[row,2] = re.sub('[0-9]','',sub_edge[row,2])+str(row+1)
            node_edge_term_cc = np.vstack((other_res,sub_edge))
        self.replaced_tn_te_cc = node_edge_term_cc
    def write_view_replaced(self,gro):
        tempgro(gro,self.replaced_tn_te_cc)
        viewgro(gro)
        

In [ ]:
import datetime
def write_cif_nobond(placed_all, params, cifname,unit_cell):
	a,b,c,alpha,beta,gamma = params
	os.makedirs('output_cifs',exist_ok=True)

	opath = os.path.join('output_cifs', cifname)
	
	with open(opath, 'w') as new_cif:
		new_cif.write('data_' + cifname[0:-4] + '\n')
		new_cif.write('_audit_creation_date              ' + datetime.datetime.today().strftime('%Y-%m-%d') + '\n')
		new_cif.write("_audit_creation_method            'MOFbuilder_1.0'" + '\n')
		new_cif.write("_symmetry_space_group_name_H-M    'P1'" + '\n')
		new_cif.write('_symmetry_Int_Tables_number       1' + '\n')
		new_cif.write('_symmetry_cell_setting            triclinic' + '\n')
		new_cif.write('loop_' + '\n')
		new_cif.write('_symmetry_equiv_pos_as_xyz' + '\n')
		new_cif.write('  x,y,z' + '\n')
		new_cif.write('_cell_length_a                    ' + str(a) + '\n')
		new_cif.write('_cell_length_b                    ' + str(b) + '\n')
		new_cif.write('_cell_length_c                    ' + str(c) + '\n')
		new_cif.write('_cell_angle_alpha                 ' + str(alpha) + '\n')
		new_cif.write('_cell_angle_beta                  ' + str(beta) + '\n')
		new_cif.write('_cell_angle_gamma                 ' + str(gamma) + '\n')
		new_cif.write('loop_' + '\n')
		new_cif.write('_atom_site_label' + '\n')
		new_cif.write('_atom_site_type_symbol' + '\n')
		new_cif.write('_atom_site_fract_x' + '\n')
		new_cif.write('_atom_site_fract_y' + '\n')
		new_cif.write('_atom_site_fract_z' + '\n')
		

		for line in placed_all:
			atom = line[0]
			f_coords = list(map(float, line[1:4]))
			c_coords = np.dot(np.linalg.inv(unit_cell), f_coords)
			new_cif.write('{:7} {:>4} {:>15.10f} {:>15.10f} {:>15.10f}'.format(
				atom, 
				re.sub('[0-9]', '', atom),  #atom type
				np.round(c_coords[0], 10), 
				np.round(c_coords[1], 10), 
				np.round(c_coords[2], 10)))
			new_cif.write('\n')

		new_cif.write('loop_' + '\n')



In [9]:
sorted_nodes = fcu.sorted_nodes
optimized_pair = fcu.optimized_pair
node_atom = fcu.node_atom
linker_atom = fcu.linker_atom
linker_x_ccoords = fcu.linker_x_ccoords
linker_ccoords = fcu.linker_ccoords
scaled_rotated_xxxx_positions = fcu.scaled_rotated_xxxx_positions
scaled_rotated_chain_node_positions = fcu.scaled_rotated_node_positions
sc_unit_cell = fcu.sc_unit_cell
optimized_params = fcu.optimized_params
sG = fcu.sG

placed_node,placed_edge = place_edgeinnodeframe(sorted_nodes,optimized_pair,node_atom,linker_atom,linker_x_ccoords,linker_ccoords,scaled_rotated_xxxx_positions,scaled_rotated_chain_node_positions)



# Save results to XYZ
placed_all = []
with open("placed_structure.xyz", "w") as file:
    #node_num_atoms = sum(len(positions) for positions in scaled_rotated_chain_node_positions.values())
    node_num_atoms = sum(len(positions) for positions in placed_node.values())
    edge_num_atoms = sum(len(positions) for positions in placed_edge.values())
    num_atoms = node_num_atoms + edge_num_atoms
    file.write(f"{num_atoms}\n")
    file.write("Optimized structure\n")
    node_idx = 1
    edge_idx = -2
    for node, positions in placed_node.items():
        if 'DV' in G.nodes[node]['type']:
            continue
        for pos in positions:
            file.write(f"{pos[0]}   {pos[1]:.8f} {pos[2]:.8f} {pos[3]:.8f}\n")
            #if pos[0]=='X':
            #    pos[0] = 'C'
            line = np.array([pos[0],pos[1],pos[2],pos[3],0,pos[0],node_idx,'NODE'])
            placed_all.append(line)
        node_idx += 1

    for edge,positions in placed_edge.items():
        
        if 'DE' in G.edges[edge]['type']:
           # pass
            continue
        for pos in positions:
            file.write(f"{pos[0]}   {pos[1]:.8f} {pos[2]:.8f} {pos[3]:.8f}\n")
            #if pos[0]=='X':
               # pos[0] = 'C'
            line = np.array([pos[0],pos[1],pos[2],pos[3],0,pos[0],edge_idx,'EDGE'])
            placed_all.append(line)
        edge_idx -= 1
            #placed_all.append(pos)





placed_all = np.array(placed_all)

write_cif_nobond(placed_all, optimized_params, 'placed_structure.cif',sc_unit_cell)

placed_N = placed_all[placed_all[:,7]=='NODE']
placed_E = placed_all[placed_all[:,7]=='EDGE']

n_term_file = 'methyl.pdb'
e_termfile = 'CCO2.pdb'

supercell = [0,0,0]

mof = MOF_ditopic()
mof.node_topics = 2
mof.load(sG,placed_all,sc_unit_cell,placed_N,placed_E)
mof.basic_supercell(np.asarray(supercell),term_file =n_term_file,boundary_cut_buffer = 0.2,edge_center_check_buffer = 0.3,cutxyz=[True,True,True])
mof.write_basic_supercell('20test.gro','20test.xyz')
mof.node_topics = 2
mof.defect_missing()
mof.term_defective_model(n_term_file=n_term_file,e_termfile=e_termfile)
mof.write_tntemof('30.gro')    
#  
#'''cleave all unsaturated linkers'''
u_edge_idx=[]
for i in mof.unsaturated_main_frag_edges:
    u_edge_idx.append(int(i[0][1:]))
print(u_edge_idx)

mof.defect_missing([],u_edge_idx)#+[26,20,18,30,8,34,33,4,28,29])
mof.term_defective_model(n_term_file=n_term_file,e_termfile=e_termfile)
mof.write_tntemof('33.gro') 
 

row_diff_idx_loose [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

this MOF has 1 fragment
UNsaturated edges(linkers) exist, need linker_termination <= 54
only saturated nodes exist
degree of edge E6 is 1, neighbor nodes are [1]
degree of edge E7 is 1, neighbor nodes are [1]
degree of edge E8 is 1, neighbor nodes are [1]
degree of edge E9 is 1, neighbor nodes are [1]
degree of edge E10 is 1, neighbor nodes are [1]
degree of edge E11 is 1, neighbor nodes are [1]
degree of edge E12 is 1, neighbor nodes are [1]
degree of edge E13 is 1, neighbor nodes are [1]
degree of edge E14 is 1, neighbor nodes are [1]
degree of edge E18 is 1, neighbor nodes are [2]
degree of edge E19 is 1, neighbor nodes are [2]
degree of edge E20 is 1, neighbor nodes are [2]
degree of edge E21 is 1, neighbor nodes are [2]
degree of edge E23 is 1, neighbor nodes are [3]
degree of edge E24 is 1, neighbor nodes are [3]
degree of edge E26 is 1, neighbor nodes are [7]
degree of edge E27 is 1, neighbor nodes are [6]
degree of edge E29 is 1, neighbor nodes are [10]
degree of edge E30 is 1,

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

[6, 7, 8, 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 23, 24, 26, 27, 29, 30, 31, 32, 33, 34, 36, 37, 39, 41, 43, 45, 48, 51, 52, 53, 58, 59, 62, 63, 65, 67, 68, 71, 72, 73, 74, 75, 76, 78, 80, 82, 84, 85, 86, 88, 90]
this MOF has 1 fragment
only saturated edges(linkers) exist
only saturated nodes exist


3Dmol.js failed to load for some reason. Please check your browser console for error messages.